In [2]:
import pandas as pd

### Prepare Data

Investigating January 2019 month data

In [3]:
csv_file_path = "/Users/sirvartsarafian/Bikes/Data/All Data/JC-201901-citibike-tripdata.csv"
df = pd.read_csv(csv_file_path)

# Convert time to datetime
df['starttime'] = pd.to_datetime(df['starttime'])
#df = df[(df['starttime'].dt.day <=30)]
df = df[df['starttime'].dt.weekday < 5]

print("There are " + str(df.shape[0]) + " journeys and " + str(df.shape[1]) + " columns.")
df.head(5)

There are 16409 journeys and 15 columns.


,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,201,2019-01-01 03:09:09.711,2019-01-01 03:12:30.8790,3183,Exchange Place,40.716247,-74.033459,3214,Essex Light Rail,40.712774,-74.036486,29612,Subscriber,1993,1
1,505,2019-01-01 05:18:00.106,2019-01-01 05:26:25.9050,3183,Exchange Place,40.716247,-74.033459,3638,Washington St,40.724294,-74.035483,29213,Subscriber,1972,2
2,756,2019-01-01 10:36:33.340,2019-01-01 10:49:10.2600,3183,Exchange Place,40.716247,-74.033459,3192,Liberty Light Rail,40.711242,-74.055701,26164,Subscriber,1985,1
3,1575,2019-01-01 12:43:38.643,2019-01-01 13:09:54.5280,3183,Exchange Place,40.716247,-74.033459,3638,Washington St,40.724294,-74.035483,29672,Customer,1969,0
4,1566,2019-01-01 12:43:39.601,2019-01-01 13:09:46.5100,3183,Exchange Place,40.716247,-74.033459,3638,Washington St,40.724294,-74.035483,29522,Customer,1969,0


### Find unique stations and their coordinates

In [4]:
# Extract unique stations and their coordinates
unique_stations = df[['start station name', 'start station latitude', 'start station longitude']].drop_duplicates()

# Rename columns for consistency
unique_stations.columns = ['station name', 'latitude', 'longitude']
unique_stations = unique_stations.reset_index(drop=True)
print("There are " + str(unique_stations.shape[0]) + " unique stations.")
unique_stations.head(5)

There are 51 unique stations.


,station name,latitude,longitude
0,Exchange Place,40.716247,-74.033459
1,Paulus Hook,40.714145,-74.033552
2,City Hall,40.717732,-74.043845
3,Grove St PATH,40.719586,-74.043117
4,Warren St,40.721124,-74.038051


### For each station, count how many bikes left from and arrived to it

In [5]:
# Calculate the count of bikes starting from each station
start_counts = df.groupby(['start station name', 'start station latitude', 'start station longitude']).size().reset_index(name='count start')

# Calculate the count of bikes ending at each station
end_counts = df.groupby(['end station name']).size().reset_index(name='count end')

# Merge the two dataframes on station name
merged_counts = pd.merge(start_counts, end_counts, left_on='start station name', right_on='end station name', how='inner')

# Calculate the percentage difference in counts
merged_counts['percentage diff'] = ((merged_counts['count end'] - merged_counts['count start']) / (merged_counts['count end']+merged_counts['count start'])) * 100
merged_counts['total count'] = (merged_counts['count start'] + merged_counts['count end'])

# Display the result including counts for bikes starting and ending
result_df = merged_counts[['start station name', 'start station latitude', 'start station longitude', 'count start', 'count end', 'percentage diff', 'total count']]
result_df.columns = ['station name', 'station latitude', 'station longitude', 'count start', 'count end', 'percentage diff', 'total count']
sorted_result = result_df.sort_values(by='total count', ascending=False).reset_index(drop=True)

# Display the result
print(sorted_result.shape) # check there are 51 unique stations
sorted_result.head(10)

(51, 7)


,station name,station latitude,station longitude,count start,count end,percentage diff,total count
0,Grove St PATH,40.719586,-74.043117,2366,2587,4.461942,4953
1,Sip Ave,40.730743,-74.063784,964,921,-2.281167,1885
2,Hamilton Park,40.727596,-74.044247,837,910,4.178592,1747
3,Exchange Place,40.716247,-74.033459,737,867,8.104738,1604
4,Newport PATH,40.727224,-74.033759,700,656,-3.244838,1356
5,Harborside,40.719252,-74.034234,590,629,3.199344,1219
6,City Hall,40.717732,-74.043845,424,462,4.288939,886
7,Newark Ave,40.721525,-74.046305,426,436,1.160093,862
8,Brunswick & 6th,40.726012,-74.050389,406,405,-0.123305,811
9,Jersey & 3rd,40.723332,-74.045953,430,366,-8.040201,796


### Count the frequency of each individual bike route 

In [6]:
bike_routes_count = df.groupby(['start station name', 'end station name', 'start station latitude', 'start station longitude', 'end station latitude', 'end station longitude']).size().reset_index(name='count')
bike_routes_count.sort_values(by='count', ascending=False, inplace=True)
bike_routes_count = bike_routes_count.reset_index(drop=True)
bike_routes_count.head(5)

,start station name,end station name,start station latitude,start station longitude,end station latitude,end station longitude,count
0,Hamilton Park,Grove St PATH,40.727596,-74.044247,40.719586,-74.043117,367
1,Grove St PATH,Hamilton Park,40.719586,-74.043117,40.727596,-74.044247,363
2,Brunswick St,Grove St PATH,40.724176,-74.050656,40.719586,-74.043117,235
3,Jersey & 6th St,Grove St PATH,40.725289,-74.045572,40.719586,-74.043117,225
4,Brunswick & 6th,Grove St PATH,40.726012,-74.050389,40.719586,-74.043117,221


### Sum routes going both directions

In [8]:
import numpy as np
summed_routes_df = bike_routes_count.copy()

# Sort start station and end station columns to create a common representation for both directions
summed_routes_df[['start station name', 'end station name']] = \
    pd.DataFrame(np.sort(bike_routes_count[['start station name', 'end station name']], axis=1), index=bike_routes_count.index)

# Group by the sorted start station and end station, and sum the counts
summed_routes_df = summed_routes_df.groupby(['start station name', 'end station name']).agg({
    'count': 'sum',
    'start station latitude': 'first',
    'start station longitude': 'first',
    'end station latitude': 'first',
    'end station longitude': 'first'
})

# Display the new DataFrame
summed_routes_df.sort_values(by='count', ascending=False, inplace=True)
summed_routes_df = summed_routes_df.reset_index()
summed_routes_df.head()

,start station name,end station name,count,start station latitude,start station longitude,end station latitude,end station longitude
0,Grove St PATH,Hamilton Park,730,40.727596,-74.044247,40.719586,-74.043117
1,Brunswick & 6th,Grove St PATH,422,40.726012,-74.050389,40.719586,-74.043117
2,Brunswick St,Grove St PATH,405,40.724176,-74.050656,40.719586,-74.043117
3,Dixon Mills,Grove St PATH,402,40.721630,-74.049968,40.719586,-74.043117
4,Grove St PATH,Jersey & 6th St,383,40.725289,-74.045572,40.719586,-74.043117
